In [1]:
import requests
import json

In [2]:
user = {'username': 'administrator', 'email': 'joshua.d.arcy@duke.edu','password': 'Admin10!', 'age': 26, 'activity_level': 2, 'gender_identity': 1}

## Registration

In [167]:
#create user if not already created
r = requests.post('http://0.0.0.0:5000/registration', json = user)
r.json()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Login

In [9]:

#login user
r = requests.post('http://127.0.0.1:5000/login', json = user)
print(r.content)
data = r.json()

try:
    access = data['access_token']
    refresh = data['refresh_token']
    print(data['message'])
    print('JSON Web Token: ' + access[:30])
except:
    print(data['message'])

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"\n  "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n  <head>\n    <title>flask.cli.NoAppException: While importing &quot;run&quot;, an ImportError was raised:\n\nTraceback (most recent call last):\n  File &quot;/Users/joshuadarcy/miniconda3/lib/python3.7/site-packages/flask/cli.py&quot;, line 240, in locate_app\n    __import__(module_name)\n  File &quot;/Users/joshuadarcy/Documents/nutrics/run.py&quot;, line 9, in &lt;module&gt;\n    from flask_restful import Api #API organization\nModuleNotFoundError: No module named \'flask_restful\' // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css"\n        type="text/css">\n    <!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. -->\n    <link rel="shortcut icon"\n        href="?__debugger__=

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Refresh Token

In [163]:
headers = { 'Authorization' : 'Bearer ' + refresh}

r = requests.post('http://127.0.0.1:5000/token/refresh', headers=headers)
data = r.json()
access = data['access_token']
print(data['message'])

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

## Admin Endpoint

In [17]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5000/admin/food/dump', headers=headers)
r.json()

{'message': []}

## Edit preferences

In [21]:
headers = { 'Authorization' : 'Bearer ' + access}

data = {'preference': 'kale salad', 
        'preference_action': 'add'}

r = requests.post('http://127.0.0.1:5000/editpreferences', json = data, headers = headers)
r.json()

{'message': 'kale salad successfully added for administrator.'}

## List preferences

In [25]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.get('http://127.0.0.1:5000/preferences', headers = headers)
r.json()['message']

[{'username': 'administrator',
  'preference': 'kale salad',
  'added': '2020-05-07 17:45:54.503389'}]

## Get Recommendation

In [145]:
headers = { 'Authorization' : 'Bearer ' + access}
location = {'latitude': 44, 'longitude': -79, 'distance': 1000}

r = requests.post('http://127.0.0.1:5000/recommendation', headers = headers, json=location)

try:
    for i in json.loads(r.json()['data']):
        print('-----------------------------------------------------------')
        print('Based on: ' + user['username'] + '\'s preference for ' + i['based_on'])
        print('-----------------------------------------------------------')
        for item in i:
            print(f'{item}: {i[item]} \n')
except: 
    print(r.json())

-----------------------------------------------------------
Based on: administrator's preference for kale salad
-----------------------------------------------------------
item name: juice box 

based_on: kale salad 

description: Its secretly a salad. 

restaurant: Local bar 

price: 2.3 

location: [44.0, -79.0] 

distance: 0.0 

match to kale salad (previous item): 0.9341701662020266 

nutrition score: -3 

recommended nutrition: {"calories": 600.0, "sugar": 54.0, "total_fats": 162.00000000000003, "sat_fats": 36.0, "cholesterol": 50.0, "carbohydrates": 330.00000000000006, "protein": 15.333333333333334, "fiber": 7.466666666666666, "sodium": 766.6666666666666, "magnesium": 106.66666666666667, "potassium": 1566.6666666666667, "calcium": 416.6666666666667} 

actual nutrition: {"magnesium": 20, "calcium": 25} 

embedding: [0.20154948718845844, 0.23743889946490526, 0.3042665123939514, 0.3846535310149193, 0.3066305108368397, 0.5105210095643997, 0.40845271199941635, 0.4470686614513397, 0.34

## Logout

In [10]:
headers = { 'Authorization' : 'Bearer ' + access}
r = requests.post('http://127.0.0.1:5001/logout/access', headers = headers)
r.json()

{'message': 'Access token has been revoked. You will be unable to use this token in the future.'}

## Add food option

In [23]:
import csv

In [12]:
with open('/Users/joshuadarcy/Documents/nutrics/development/food.csv', newline='') as f:
    reader = csv.reader(f)
    skip_first = True
    for row in reader:
        try:
            if skip_first == True:
                skip_first = False
                continue
            a = row[0].split('|')
            toadd = {
                'item_name' : a[0],
                'latitude' : float(a[1]),
                'longitude' : float(a[2]),
                'restaurant_name' : a[3],
                'item_description' : 'shhh--secret',
                'price' : 1.50
            }
            r = requests.post('http://127.0.0.1:5000/admin/food/add', headers = headers, json = toadd)
        except:
            continue

FileNotFoundError: [Errno 2] No such file or directory: '/Users/joshuadarcy/Documents/nutrics/development/food.csv'

In [29]:
import csv
import json

In [47]:
headers = {'Authorization' : 'Bearer ' + access}
toadd = {
'item_name' : 'juice box',
'latitude': 44, 
'longitude': -79,
'restaurant_name' : 'Local bar',
'item_description' : 'Its secretly a salad.',
'nutrition' : json.dumps({'magnesium': 20, 'calcium': 25}),
'price' : 2.30
}

r = requests.post('http://127.0.0.1:5000/admin/food/add', headers = headers, json = toadd)
r.json()

{'message': 'juice box @ 44,-79 added.'}

In [37]:
headers = {'Authorization' : 'Bearer ' + access}
toedit = {
    'item_name' : 'juice',
    'latitude': 44, 
    'longitude': -78,
    'restaurant_name' : 'Local bar',
    'item_description' : 'Its totally a juice.',
    'price' : 2.30
    }
r = requests.post('http://127.0.0.1:5000/admin/food/edit', headers = headers, json = toedit)
r.json()

{'message': 'juice updated.'}

In [115]:
class Nutrition_Score: 

    def __init__(self, age, gender_identity, activity, jstring):
        
        self.age = age
        self.gender = gender_identity
        self.activity = activity

        self.recommended = self.recommended_dict()
        self.score = self.nutrientScore(jstring, self.recommended)

    def nutrientScore(self, nutrition_info, recommended_dict_i):
        nutrition_info = json.loads(nutrition_info)
        recDict = recommended_dict_i

        nutrient_fracs={}
        for nutrient in nutrition_info:
            goal = recDict[nutrient]
            actual = nutrition_info[nutrient]
            frac = actual/goal
            nutrient_fracs[nutrient] = frac

        # select weights for each adequacy and moderation for each nutrient
        weight_adequacy, weight_moderation = self.weight(recDict)

        # Scoring
        nutrient_scores_adequacy= {}
        nutrient_scores_moderation = {}
        for nutrient in nutrient_fracs:
            if nutrient_fracs[nutrient] >= 2 :
                nutrient_scores_adequacy[nutrient] = 3* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = -3* weight_moderation[nutrient]
            if nutrient_fracs[nutrient] >= 1.25 and nutrient_fracs[nutrient] <2:
                nutrient_scores_adequacy[nutrient] = 2* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = -2* weight_moderation[nutrient]
            if nutrient_fracs[nutrient] >= 1 and nutrient_fracs[nutrient] <1.25:
                nutrient_scores_adequacy[nutrient] = 1* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = -1* weight_moderation[nutrient]
            if nutrient_fracs[nutrient] >= .75 and nutrient_fracs[nutrient] <1:
                nutrient_scores_adequacy[nutrient] = -1* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = 1* weight_moderation[nutrient]
            if nutrient_fracs[nutrient] >= .5 and nutrient_fracs[nutrient] <.75:
                nutrient_scores_adequacy[nutrient] = -2* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = 2* weight_moderation[nutrient]
            if nutrient_fracs[nutrient] >= 0 and nutrient_fracs[nutrient] <.5:
                nutrient_scores_adequacy[nutrient] = -3* weight_adequacy[nutrient]
                nutrient_scores_moderation[nutrient] = 3* weight_moderation[nutrient]
        adequacy = sum(nutrient_scores_adequacy.values())
        moderation = sum(nutrient_scores_moderation.values())
        score = adequacy + moderation # a more positive score is better
        return score

    # Generate the recommended nutritional intake depending on gender and activity
    #---------------------------------------------------------------------------------------------

    def recommended_dict(self):
        gender = self.gender
        age = self.age
        activity = self.activity

        if gender == 0:
            if activity == 0:
                if  age >=16 and age <=18:
                    cals = 2400;percent_totfat = .3;protein = 52;fiber = 30.8
                elif age >=19 and age <=20:
                    cals = 2600;percent_totfat = .27;protein = 56;fiber = 33.6
                elif age >=21 and age <=30:
                    cals = 2400;percent_totfat = .27;protein = 56;fiber = 33.6
                elif  age >=31 and age <=40:
                    cals = 2400;protein = 56;percent_totfat = .27;fiber = 30.8
                elif  age >=41 and age <=50:
                    cals = 2200;percent_totfat = .27;protein = 56;fiber = 30.8
                elif  age >=51 and age <=60:
                    cals = 2200;percent_totfat = .27;protein = 56;fiber = 28
                elif  age >=61:
                    cals = 2000;percent_totfat = .27;protein = 56;fiber = 28
            elif activity == 1:
                if  age >=16 and age <=25:
                    cals = 2800;percent_totfat = .3;protein = 52
                    if age >=16 and age <= 18:
                        fiber = 30.8
                    elif age >=19 and age <= 25:
                        fiber = 33.6
                elif age >=26 and age <=45:
                    cals = 2600;percent_totfat = .27;protein = 56
                    if age >= 26 and age <=30:
                        fiber = 33.6
                    elif age >= 31 and age <= 45:
                        fiber = 30.8
                elif age >=46 and age <= 65:
                    cals = 2400;percent_totfat = .27;protein = 56
                    if age >= 46 and age <= 50:
                        fiber = 30.8
                    elif age >=51:
                        fiber = 28
                elif age >=66:
                    cals = 2200;percent_totfat = .27;protein = 56; fiber=28
            elif activity == 2:
                if  age >=16 and age <=18:
                    cals = 3200;percent_totfat = .3;protein = 52;fiber = 30.8
                elif  age >=19 and age <=35:
                    cals = 3000;percent_totfat = .27;protein = 56
                    if age >= 19 and age <= 30:
                        fiber = 33.6
                    elif age >= 31 and age >= 35:
                        fiber = 30.8
                elif  age >=36 and age <=55:
                    cals = 2800;percent_totfat = .27;protein = 56
                    if age >= 36 and age <= 50:
                        fiber = 30.8
                    elif age >=51 and age <= 55:
                        fiber = 28
                elif  age >=56 and age <=75:
                    cals = 2600;percent_totfat = .27;protein = 56;fiber = 28
                elif age >= 76:
                    cals = 2400;percent_totfat = .27;protein = 56;fiber = 28

            recommended_dict = {'calories':cals/3,'sugar':.09*cals/3,'total_fats': percent_totfat*cals/3,'sat_fats':.06*cals/3,'cholesterol':150/3,'carbohydrates':.55*cals/3,'protein':protein/3,'fiber':fiber/3,'sodium':2300/3,'magnesium':420/3,'potassium':4700/3,'calcium':1250/3} #'totfolate':400/3,'transfat':.01*cals/3,

        elif gender == 1:
            if activity == 0:
                if age >=16 and age <=18:
                    cals = 1800;percent_totfat = .3;fiber = 25.2
                elif  age >=19 and age <=25:
                    cals = 2000;percent_totfat = .27;fiber = 28
                elif  age >=26 and age <=50:
                    cals = 1800;percent_totfat = .27
                    if age >= 26 and age <=30:
                        fiber = 28
                    elif age >30 and age <=50:
                        fiber = 25.2
                elif  age >=51:
                    cals = 1600;percent_totfat = .27;fiber = 22.4
            elif activity == 1:
                if age >=16 and age <=18:
                    cals = 2000;percent_totfat = .3;fiber = 25.2
                elif  age >=19 and age <=25:
                    cals = 2200;percent_totfat = .27;fiber = 28
                elif  age >=26 and age <=50:
                    cals = 2000;percent_totfat = .27
                    if age >= 26 and age <=30:
                        fiber = 28
                    elif age >30 and age <=50:
                        fiber = 25.2
                elif  age >=51:
                    cals = 1800;percent_totfat = .27;fiber = 22.4
            elif activity == 2:
                if age >=16 and age <=30:
                    cals = 2400;percent_totfat = .27
                    if age >=16 and age <=18:
                        fiber = 25.2
                    elif age >18 and age <=25:
                        fiber = 28
                    elif age >= 26 and age <=30:
                        fiber = 28
                elif age >30 and age <=60:
                    cals = 2200;percent_totfat = .27
                    if age >30 and age <=50:
                        fiber = 25.2
                    elif age> 50:
                        fiber = 22.4
                elif age>60:
                    cals = 2000;percent_totfat = .27;fiber = 22.4

            recommended_dict = {'calories':cals/3,'sugar':.09*cals/3,'total_fats': percent_totfat*cals/3,'sat_fats':.06*cals/3,'cholesterol':150/3,'carbohydrates':.55*cals/3,'protein':46/3,'fiber':fiber/3,'sodium':2300/3,'magnesium':320/3,'potassium':4700/3,'calcium':1250/3} #'transfat':.01*cals/3,'totfolate':400/3,'vd':600/3,
        
        else:
            recommended_dict = {'calories':2400/3,'sugar':.09*2400/3,'total_fats': .27*2400/3,'sat_fats':.06*2400/3,'cholesterol':150/3,'carbohydrates':.55*cals/3,'protein':46/3,'fiber':25.2/3,'sodium':2300/3,'magnesium':320/3,'potassium':4700/3,'calcium':1250/3} #'totfolate':400/3,'transfat':.01*2400/3,'vd':600/3,
        
        return recommended_dict


    def weight(self, recDict):
        switcher = {
        'calories': (2,3),
        'sugar': (1,3),
        'total_fats': (1,3),
        'unsatfat': (2,2),
        'sat_fats': (1,3),
        'transfat': (1,3),
        'carbohydrates': (2,3),
        'protein': (2,3),
        'fiber': (3,1),
        'sodium': (1,3),
        'magnesium': (2,1),
        'totfolate': (2,1),
        'potassium': (2,1),
        'vd': (2,1),
        'cholesterol': (1,1), 
        'calcium': (1,1)
        }
        weight_adequacy = {} #{'nutrient': weight_value, etc}
        weight_moderation = {}
        for nutrient in recDict:
            weight_adequacy[nutrient], weight_moderation[nutrient] = switcher[nutrient]
        return weight_adequacy, weight_moderation

In [116]:
tester = json.dumps({'calories': 800.0,
 'sugar': 172.0,
 'total_fats': 216.0,
 'sat_fats': 48.0,
 'cholesterol': 50.0,
 'carbohydrates': 0.0,
 'protein': 118.666666666666668,
 'fiber': 9.333333333333334,
 'sodium': 0,
 'magnesium': 140.0,
 'potassium': 1566.6666666666667,
 'calcium': 416.6666666666667})

In [117]:
Nutrition_Score(25, 0, 0, tester).score

-5